# 데이터 로드 

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from tqdm import tqdm 
from glob import glob 
import os 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!tar -xvf '/content/drive/MyDrive/공유용/train_features.tar'

In [ ]:
csvs_dir = glob('/content/katl/*.bz2')
csvs_dir.sort()
csvs_dir = pd.DataFrame(csvs_dir,columns=['dir'])
csvs_name = list(csvs_dir['dir'].apply(lambda x : x.split('katl/')[-1].split('.csv')[0].split('_',maxsplit=1)[-1]))

In [ ]:
def make_dir(dir,df):
  csv_dir = df[df['dir'].apply(lambda x : dir in x )].values[0][0]
  return csv_dir 

In [ ]:
for csv_name in tqdm(csvs_name):
  exec(f"{csv_name} = pd.read_csv(make_dir('{csv_name}',csvs_dir))")

# CSV 별 Columns Description 

**Arrival** 

arrival_runway
- Timestamp : runway에 이륙한 시간 
- arrival_runway : 착륙한 활주로 

tfm_estimated_runway_arrival_time
- timestamp : 생성 시간 
- arrival_runway_estimated_time : 활주로에 도착 추정 시간 

scheduled_runway_arrival_time
- timestampe : 생성 시간 
- scheduled_runway_arrival_time : 비행기가 활주로에 도착 예정된 시간 (계획) 

runway_arrival_time 
- 착륙 활주로에 도달한 시간 

stand_arrival_time
- 착륙 gate에 도달한 시간 

**Departure**

departure_runway
- Timestamp : 활주로에서 출발한 시간 
- departure_runway : 착륙한 활주로 

etd : estimated departure time 
- timestamp : 생성 시간 
- estimated_runway_departure_time : 활주로에 도착 추정 시간 

stand_departure_time 
- gate에 도달한 시간 

runway_departure_time 
- 이륙 활주로에 도달한 시간 




**ETC** 


first_position
- flight가 NAS 시스템에 최초로 추적되기 시작한 시간 

Lamp 
- 환경 변수(기상) 

airport_config
- scheduled_runway_arrival_time

# Runway DF 세팅 

In [ ]:
arrival_runway = pd.read_csv(make_dir('arrival_runway',csvs_dir))
departure_runway = pd.read_csv(make_dir('departure_runway',csvs_dir))

In [ ]:
def split_timestamp(df):
  df['date'] = df['timestamp'].apply(lambda x : x.split('T')[0])
  df['hour'] = df['timestamp'].apply(lambda x : x.split('T')[1].split(':')[0])
  df['min'] = df['timestamp'].apply(lambda x : x.split('T')[1].split(':')[1])
  df['sec'] = df['timestamp'].apply(lambda x : x.split('T')[1].split(':')[2])
  return df

In [ ]:
departure_runay = split_timestamp(departure_runway)
departure_runway['D_runway']= departure_runway['departure_runway']
departure_runway = departure_runway.drop(columns = 'departure_runway')

In [ ]:
arrival_runway = split_timestamp(arrival_runway)
arrival_runway['A_runway']= arrival_runway['arrival_runway']
arrival_runway = arrival_runway.drop(columns = 'arrival_runway')
arrival_runway = arrival_runway.sort_values(by=['timestamp'])

In [ ]:
runway = pd.concat([arrival_runway,arrival_runway])
runway = runway.sort_values(by='timestamp').reset_index().drop(columns='index')
runway['timestamp'] = pd.to_datetime(runway['timestamp'])
runway['hour'] = runway['hour'].map(int)
runway['min'] = runway['min'].map(int)
runway['sec'] = runway['sec'].map(int)
print(len(runway))

664882


# Airport config 세팅

In [ ]:
airport_config = pd.read_csv(make_dir("airport_config",csvs_dir))
airport_config = split_timestamp(airport_config)
airport_config = airport_config.sort_values(by= 'timestamp').reset_index().drop(columns='index')

In [ ]:
airport_config

,timestamp,airport_config,date,hour,min,sec
0,2020-11-01T01:11:00,D_8R_9L_A_10_8L_9R,2020-11-01,01,11,00
1,2020-11-01T01:57:00,D_8R_9L_A_10_8L_9R,2020-11-01,01,57,00
2,2020-11-01T02:53:00,D_8R_9L_A_10_8L_9R,2020-11-01,02,53,00
3,2020-11-01T03:54:00,D_8R_9L_A_10_8L_9R,2020-11-01,03,54,00
4,2020-11-01T04:52:00,D_8R_9L_A_10_8L_9R,2020-11-01,04,52,00
...,...,...,...,...,...,...
12426,2021-10-31T20:56:00,D_26L_27R_A_26R_27L_28,2021-10-31,20,56,00
12427,2021-10-31T21:53:00,D_26L_27R_A_26R_27L_28,2021-10-31,21,53,00
12428,2021-10-31T22:04:00,D_26L_27R_A_26R_27L_28,2021-10-31,22,04,00
12429,2021-10-31T22:54:00,D_26L_27R_A_26R_27L_28,2021-10-31,22,54,00


In [ ]:
date = '2020-11-01'
hour = 0 
init = 30
fin = 60
std = (runway['date']==date)&(runway['hour']==hour)&(runway['min']>=init)&(runway['min']<fin)
temp = runway[std]
A_runway = list(set(temp[temp['D_A']=='A']['runway']))
D_runway = list(set(temp[temp['D_A']=='D']['runway']))
print(f'D_{D_runway}_A_{A_runway}')

D_['09L']_A_['09R', '08L']


In [ ]:
date = '2020-11-01'
hour = np.arange(24)
init = [0,30]
fin = [30,60]
for hour in range(4):
  for init,fin in [0,30],[30,60]:
    std = (runway['date']==date)&(runway['hour']==hour)&(runway['min']>=init)&(runway['min']<fin)
    temp = runway[std]
    A_runway = list(set(temp[temp['D_A']=='A']['runway']))
    D_runway = list(set(temp[temp['D_A']=='D']['runway']))
    print(f'D_{D_runway}_A_{A_runway}')

D_['09L', '08R']_A_['09R', '08L']
D_['09L']_A_['09R', '08L']
D_['09L']_A_['09R']
D_['09L']_A_['09R', '08L']
D_['09L', '08R']_A_['09R', '08L']
D_['08R']_A_['09R', '08L']
D_['09L', '08R']_A_[]
D_['09L']_A_[]


In [ ]:
config = pd.DataFrame([date,hour,fin]).T
config.columns = ['date','hour','minute']

In [ ]:
D_runway

['09L']

In [ ]:
config['D_runway']=D_runway
config['A_runway']=A_runway

ValueError: ignored

In [ ]:
config

,date,hour,minute,D_runway,A_runway
0,2020-11-01,3,60,0,0
